<a href="https://colab.research.google.com/github/mcppp/Intro2DL/blob/main/Copy_of_%5BSKELETON%5D_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
### USE ONLY THESE PACKAGES ###
import os
import csv
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

### Upload the lyrics_dataset.zip in the colab's folder first !

In [4]:
!unzip lyrics_dataset.zip

Archive:  lyrics_dataset.zip
   creating: lyrics_dataset/
  inflating: lyrics_dataset/adele.txt  
  inflating: lyrics_dataset/al-green.txt  
  inflating: lyrics_dataset/alicia-keys.txt  
  inflating: lyrics_dataset/amy-winehouse.txt  
  inflating: lyrics_dataset/beatles.txt  
  inflating: lyrics_dataset/bieber.txt  
  inflating: lyrics_dataset/bjork.txt  
  inflating: lyrics_dataset/blink-182.txt  
  inflating: lyrics_dataset/bob-dylan.txt  
  inflating: lyrics_dataset/bob-marley.txt  
  inflating: lyrics_dataset/britney-spears.txt  
  inflating: lyrics_dataset/bruce-springsteen.txt  
  inflating: lyrics_dataset/bruno-mars.txt  
  inflating: lyrics_dataset/cake.txt  
  inflating: lyrics_dataset/dickinson.txt  
  inflating: lyrics_dataset/disney.txt  
  inflating: lyrics_dataset/dj-khaled.txt  
  inflating: lyrics_dataset/dolly-parton.txt  
  inflating: lyrics_dataset/dr-seuss.txt  
  inflating: lyrics_dataset/drake.txt  
  inflating: lyrics_dataset/eminem.txt  
  inflating: lyrics_data

### Do not change this seed number

In [5]:
np.random.seed(1)
torch.manual_seed(1)

In [65]:
## Use to construct dataset and dataloader!!!!
## Use below code to train your model with all lyrics
lyrics = list()
for txt_file in os.listdir('./lyrics_dataset'):
    if txt_file[0] != '.':
        target_txt = os.path.join('./lyrics_dataset', txt_file)
        f = open(target_txt, 'r')
        curr_lyrics = f.readlines()
        for i in range(len(curr_lyrics)):
            curr_lyrics[i] = curr_lyrics[i].lower()
        curr_lyrics = list(set(curr_lyrics))
        lyrics += curr_lyrics
# print(len(lyrics)) #how many lines



#1: deleting everything but alphanumerics and blank spaces
lyrics1=lyrics[:3]  #for short example
stripped = []
for i in lyrics1:
  stripped.append ( ''.join(ch for ch in i if ch.isalnum() or ch == ' ').split())

a = list()
a = stripped

type(a[0]) #checking that self.trimmed_lyrics is a list with lists inside of it
# print(a)
# print(a[0])


#2: making vocabulary of unique words
unique = list()
for x in a:  
    for i in x:
      if i not in unique:
        unique.append(i)
# print(unique)


##3: freq dict
frequency = dict()
for x in a:
  for i in x:
      if i not in frequency:
        frequency[i] = 0 
      frequency[i] += 1
# print(frequency)

##4:
idx=0 #which sentence we want
target_lyrics = a[idx] #chosen sentence separated in words
print(target_lyrics)
# print(len(target_lyrics))

random_iterator = iter(target_lyrics) #creating iterator
target_word = next(random_iterator) #randomly choose target word in sentence
# print(target_word)
target_idx = target_lyrics.index(target_word) #index of target word
# print(target_idx)

chosen_window = 3
window_len = list(range(1, chosen_window+1, 1))
print(window_len)

sample = dict()
sample['pairs'] = list()

for i in window_len:
  if (target_idx+i)<=len_word:
    sample['pairs'].append([target_idx,target_idx + i])
  if (target_idx - i) >= 0:
    sample['pairs'].append([target_idx, target_idx - i])

print(sample['pairs'])



# sample['pairs'][0] = (2,1)

# target_idx =1
# target_word = target_lyrics[target_idx]
# print(target_word)


 
##--------------write above-------------##
        # Bring out one list, from self.trimmed_lyrics. (i.e., self.trimmed_lyrics[idx])
        # Then, the list should contain the splited sentence in each lyric.
        ### i.e., self.trimmed_lyrics[idx] would look like ['hey', 'nice', 'to', 'meet', 'you']
        # (+1) randomly select the target word from self.trimmed_lyrics[idx]
        # (+4) based on selected target word, and self.window_len,
        #      generate a tuple of (target word index, context word index),
        #      and add that tuple into sample['pairs']
        ### if 'nice' is randomly chosen as the target element, and self.window_len = 1
        ### if self.vocabulary[100] == 'hey', 
        ###    self.vocabulary[500] == 'nice',
        ###    self.vocabulary[300] == 'to'
        ### sample['pairs'] should be [(500, 300), (500, 100)]







## Use below code to train your model with specific artist's lyrics
# target_txt = './lyrics_dataset/lady-gaga.txt'
# f = open(target_txt, 'r')
# lyrics = f.readlines()
# for i in range(len(lyrics)):
#     lyrics[i] = lyrics[i].lower()
# lyrics = list(set(lyrics))

# # #"lyrics" is a list of strings.
# for l in lyrics:
#     print(l)

['groove', 'in', 'the', 'name', 'of', 'love', 'listen', 'love', 'brought', 'us', 'together']
11
groove
0
[1, 2, 3]
[[3, 4], [3, 2], [3, 5], [3, 1], [3, 6], [3, 0]]


In [ ]:
#HELP EXAMPLES

print(lyrics[103])
trimmed_lyrics = list()
trimmed_lyrics.append(['chola', 'or', 'orient', 'made'])

trimmed_lyrics.append(['me', 'or', 'you'])
# print(trimmed_lyrics) #how we want it to look like (eliminate symbols too)
# vocabulary= ['chola', 'or', 'orient', 'made','me', 'you'] #vocab list with unique words

# frequency=dict() #how many times a word appears
# frequency['chola'] = 1
# frequency['or'] = 2
# frequency['orient'] = 1
# frequency['made'] = 1
# frequency['me'] = 1
# frequency['you'] = 1
# print(frequency)

#MUST DO 4
print(trimmed_lyrics[0])  #first list of words
target_lyrics = trimmed_lyrics[0]
targed_idx =1
target_word = target_lyrics[target_idx]
print(target_word)
vocabulary= ['chola', 'or', 'orient', 'made','me', 'you'] #vocab list with unique words
    #          0        1       2        3     4     5
sample = dict()
sample['pairs'] = list()
sample['pairs'] = [(1,2),(1,0)] #[('or','orient'),('or', 'chola')] #  needs to be converted into integer
                 #(targe word's index, context word's index)
#your code here


chola or orient made

['chola', 'or', 'orient', 'made']


In [ ]:
#we wanna get lyrics and eliminate commas, brackets, etc.

class lyricsDataset(Dataset): # 12 points
    def __init__(self, lyrics, window_len):
        self.lyrics = lyrics
        self.window_len = window_len #how many words we see before and after target word
        # window len means, that, if i-th word is target
        # you would consider i-window_len ~ i+window_len words as context words

        ##--------------write below-------------## <--- do not erase this afterwards
        self.trimmed_lyrics = list()    
        self.vocabulary = list()
        self.frequency = dict()
        # your code in here
        
        #1: Eliminating everything but alphanumerics and blank spaces + splitting by blank spaces
        stripped = []
        for i in lyrics:
          stripped.append ( ''.join(ch for ch in i if ch.isalnum() or ch == ' ').split())
        self.trimmed_lyrics = stripped

        #2: Making vocabulary of unique words
        unique = list()
        for x in self.trimmed_lyrics:  
          for i in x:
            if i not in unique:
              unique.append(i)
        self.vocabulary = unique

        #3: Creating frequency dict
        for x in self.trimmed_lyrics:
          for i in x:
            if i not in self.frequency:
              self.frequency[i] = 0 
            self.frequency[i] += 1
        
    
        ##--------------write above-------------## <--- do not erase this
        
        ### MUST TO-DO 1. (+3) -> self.trimmed_lyrics
        # for strings in self.lyrics, 
        # (+2) write a code to eliminate every character except for alphabet, number, and space (" ")
        ### for example, - + ? ! ' " [ ] ( ) <- char like this should be excluded. 
        # (+1) after that, split each string with respect to the space.
        ### If proprocessed string is "hello hello hello", 
        ### a list ['hello', 'hello', 'hello'] should be generated.
        ### Then, put that list into the self.trimmed_lyrics.
        ### self.trimmed_lyrics needs to be a LIST which has LIST as element.

        ### MUST TO-DO 2. (+2) -> self.vocabulary
        # (+2) Put all words(string) in self.trimmed_lyrics to the self.vocabulary. 
        ### In self.vocabulary, each word needs to be unique.
        ### which means, this list (self.vocabulary) should not have duplicated elements. 
        ### self.vocabulary needs to be a LIST which contains unique words in self.trimmed_lyrics
        ### If your code contains duplicated words, you would not get the point.
        ### If there is a word that are neglected from self.trimmed_lyrics, you would not get the point.

        ### MUST TO-DO 3. (+2) -> self.frequency
        # (+2) In self.frequency, Record how many times each word in self.vocabulary 
        ###                              appears in self.trimmed_lyrics.
        ### For example, if "love" appears 100 times in self.trimmed_lyrics,
        ### it should be: self.frequency["love"] = 100

    def __len__(self):
        # DO NOT TOUCH BELOW. JUST USE BELOW CODE FOR YOUR __len__ 
        return len(self.trimmed_lyrics)
        # DO NOT TOUCH ABOVE. JUST USE ABOVE CODE FOR YOUR __len__ 

    def __getitem__(self, idx):
        ### MUST TO-DO 4. (+5) --> sample (bring out positive samples from lyrics inside this dataset)
        ##--------------write below-------------##
        sample = dict()
        sample['pairs'] = list()
        # your code here
        random_iterator = iter(self.trimmed_lyrics[idx]) #creating iterator
        target_word = next(random_iterator) #randomly choose target word in sentence
        target_idx = self.trimmed_lyrics[idx].index(target_word) #index of target word  
        
        window_len = list(range(1, self.window_len+1, 1)) #range to check if pair can be made

        for i in window_len:
          if (target_idx+i)<=len_word:
            sample['pairs'].append([target_idx,target_idx + i])
          if (target_idx - i) >= 0:
            sample['pairs'].append([target_idx, target_idx - i])

        return sample
        ##--------------write above-------------##
        # Bring out one list, from self.trimmed_lyrics. (i.e., self.trimmed_lyrics[idx])
        # Then, the list should contain the splited sentence in each lyric.
        ### i.e., self.trimmed_lyrics[idx] would look like ['hey', 'nice', 'to', 'meet', 'you']
        # (+1) randomly select the target word from self.trimmed_lyrics[idx]
        # (+4) based on selected target word, and self.window_len,
        #      generate a tuple of (target word index, context word index),
        #      and add that tuple into sample['pairs']
        ### if 'nice' is randomly chosen as the target element, and self.window_len = 1
        ### if self.vocabulary[100] == 'hey', 
        ###    self.vocabulary[500] == 'nice',
        ###    self.vocabulary[300] == 'to'
        ### sample['pairs'] should be [(500, 300), (500, 100)]

In [ ]:
### DO NOT TOUCH BELOW. JUST USE THESE LINES.
### PENALTY (-5) CAN BE APPLIED IF YOUR CODE DOES NOT WORK FOR VARIOUS VALUES OF WINDOW_LEN
dataset = lyricsDataset(lyrics, 3)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
### DO NOT TOUCH ABOVE. JUST USE THESE LINES.

118784


118784

In [ ]:
class Word2Vec(nn.Module): # 15 points
    def __init__(self, num_vocabs, embed_dim = 300): ## do not change 'embed_dim' value.
        ### MUST TO-DO 5 : (+5)
        ##--------------write below-------------##
        # your code here
        ##--------------write above-------------##
        # Define a model which can map word's integer index value to word embedding.
        
    def forward(self, pairs):
        '''
        input : pairs - one tuple of (target word index, context word index)
        '''
        ### MUST TO DO 6 : (+10)
        ##--------------write below-------------##
        # your code here
        ##--------------write above-------------##
        # return the embedding of target word and context word.
        # i.e., return target_embed, context_embed


In [ ]:
### DO NOT TOUCH BELOW. JUST USE THESE LINES. MAKE YOUR CODE WORK WITH THESE LINES
model = Word2Vec(len(dataset.vocabulary)) #the arg is num_vocabulary
optimizer = torch.optim.AdamW(model.parameters())
device = 'cuda'
### DO NOT TOUCH ABOVE. JUST USE THESE LINES. MAKE YOUR CODE WORK WITH THESE LINES
### USE THIS GIVEN OPTIMIZER.

In [ ]:
def train(model, optimizer, sample_1, sample_2): # 13 points
    ### MUST TO DO 6 : (+12)
    ##--------------write below-------------##
    model.train()
    model = model.to(device)
    optimizer.zero_grad()
    # your code here
    optimizer.step()
    # return the current loss value
    ##--------------write above-------------##

    ## sample_1 will contain positive (target_1, context_1) tuples
    ## sample_2 will contain another positive (target_2, context_2) tuples
    ## But for the negative sampling, we need negative (target, context) tuples.
    ## NEGATIVE TUPLES can be generated by (target_1, context_2), (target_2, context_1)
    ## (+5) Generate and give positive & negative tuples for model's input
    ##### Then, you would get : target_embed, context_embed = model((target, context)).
    ## (+4) calculate the distance between target_embed and context_embed by DOT PRODUCT  #see slides
    ## (+4) calculate the loss based on that distance, and optimize the model
    ####    Label positive tuples as class '1', otherwise as class '0'
    ####    You can also use sigmoid function rather than the softmax function.
    ####    At the end, you must return the current loss value.

In [ ]:
### DO NOT TOUCH BELOW. JUST USE THESE LINES. MAKE YOUR CODE WORK THESE CODES.
### PENALTY (-5) WILL BE GIVEN WHEN YOUR CODE RAISES AN ERROR DURING EPOCH.
### YOUR TRAINING NEEDS TO BE PERFORMED WITH VARIOUS .TXT FILES.
max_epoch = 1
for epoch in range(max_epoch):
    total_loss = 0.0
    cnt = 0
    for sample in tqdm(dataloader):
        if cnt > 0:
            curr_loss = train(model, optimizer, sample, prev_sample)
            total_loss += curr_loss / len(dataloader)    
        prev_sample = sample
        cnt += 1
        if cnt % 200 == 0:
            print('[EPOCH {}] SAMPLED TRAIN LOSS : {}'.format(epoch, curr_loss))
    print('[EPOCH {}] TOTAL LOSS : {}'.format(epoch, total_loss))

In [ ]:
### MUST TO DO 7 : (+10)
##--------------write below-------------##
# your code here
##--------------write above-------------##
## (+10) bring your word embedding as a numpy array "embedding"
#### "embedding" should be N by D array, 
#### where N is the number of vocabularies, and D is the dimension of the word embedding. D is 300 in this code
#### embedding[i, :] should be word embedding of dataset.vocabulary[i]

In [ ]:
### JUST USE THESE LINES. MAKE YOUR CODE WORK THESE CODES. (check if visual makes sense , see graph in slides )
reducer = PCA(n_components=2)
# or try use
# reducer = TSNE(n_components=2, verbose=1)
reduce_results = reducer.fit_transform(embedding)

In [ ]:
### DO NOT TOUCH BELOW. JUST USE THESE LINES. MAKE YOUR CODE WORK THESE CODES.
top_k = 50
sort_idx = np.argsort(list(dataset.frequency.values()))[::-1]
sort_idx = sort_idx[:top_k]
frequent_vocabs = [list(dataset.frequency.keys())[si] for si in sort_idx]
plt.figure(figsize=(10, 6))

for idx, vocab in enumerate(dataset.vocabulary):
    if vocab in frequent_vocabs:
        plt.plot(reduce_results[idx, 0], reduce_results[idx, 1], '.')
        plt.text(reduce_results[idx, 0], reduce_results[idx, 1], vocab)\

In [ ]:
### DO NOT TOUCH BELOW. JUST USE THESE LINES. MAKE YOUR CODE WORK THESE CODES.
min_dist = 10000000
target_word = 'i'
for idx, vocab in enumerate(dataset.vocabulary):
    if vocab != target_word:
        distance = np.linalg.norm(embedding[dataset.vocabulary.index(target_word)] - embedding[idx])
        min_dist = min(distance, min_dist)
        if min_dist == distance:
            nearest_to_target = vocab
print('"{}" is nearest to "{}"'.format(target_word, nearest_to_target))